<a href="https://colab.research.google.com/github/kaustubhduddala/alzdetectionproject/blob/main/ISM_Dementia_Predictor_BETA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.2 MB/s eta 0:00:00


In [2]:
import anvil.server

anvil.server.connect("server_6VYQ7FW7WMBTMGNGGW2TKBFB-N6VO44IICV7SAHDS")
# if there is no output, there is an error

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [3]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [4]:
# Loading the dataset
# using this dataset https://www.kaggle.com/datasets/jboysen/mri-and-alzheimers/data
# I've found that you can refer to this as well for more info https://www.oasis-brains.org/files/oasis_cross-sectional_facts.pdf
data = pd.read_csv('sample_data/oasis_cross-sectional.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sample_data/oasis_cross-sectional.csv'

In [ ]:
# Data preprocessing and replace missing values
data.fillna(data.mean(numeric_only=True), inplace=True)

In [ ]:
# Encode categorical data
label_encoder = LabelEncoder()
data['M/F'] = label_encoder.fit_transform(data['M/F'])
""" data['Hand'] = label_encoder.fit_transform(data['Hand']), there are no left-handed people in this dataset, so I excluded the parameter altogether """
print(data['M/F']) #as you can see with the print, F=0 and M=1

In [ ]:
# Defining features (X) and the target (y)
X = data[['M/F', 'Age', 'Educ', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF']]
y = data['CDR']

"""
Clarifications for later:
M/F: Gender
Age: Age
Educ: Education level (following levels of education: 1: less than high school grad., 2: high school grad., 3: some college, 4: college grad., 5: beyond college)
SES: Socioeconomic Status ()
MMSE: Mini Mental State Examination
CDR: Clinical Dementia Rating (We are trying to predict this!!) -- Ratings are assigned on a 0–5 point scale, (0 = absent; 0.5 = questionable; 1= present, but mild; 2 = moderate; 3 = severe; 4 = profound; 5 = terminal)
eTIV: Estimated Total Intracranial Volume
nWBV: Normalize Whole Brain Volume
ASF: Atlas Scaling Factor (atlas-based head size normalization technique to measure the standardized total intracranial volume for comparison, classification and predication)
"""

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

In [ ]:
# Model metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

In [ ]:
# Visualize it
import matplotlib.pyplot as plt

# Creating a scatter plot to compare actual vs. predicted CDR values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, c='blue', alpha=0.6)
plt.xlabel('Actual CDR')
plt.ylabel('Predicted CDR')
plt.title('Actual vs. Predicted CDR Values')
plt.grid(True)

# Add a diagonal line to visualize perfect predictions
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', linewidth=2)

# Display the plot
plt.show()


In [ ]:
# Input your own data to get a prediction
@anvil.server.callable
def predict_cdr(model, input_data):
    """
    Parameters:
    - model: Trained regression model (e.g., Linear Regression)
    - input_data: numpy array with your data

    Returns:
    - Predicted CDR (a float)
    """
    # Ensuring that input_data has the correct number of features
    if len(input_data) != 8:
        raise ValueError("Input data should contain 8 values: 'M/F', 'Age', 'Educ', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF'")

    # Convert input_data to a NumPy array and reshape it for prediction
    input_data = np.array(input_data).reshape(1, -1)

    # Make the prediction using the trained model
    predicted_cdr = model.predict(input_data)

    return predicted_cdr[0]  # Return the predicted CDR as a float


# input_data: A list or NumPy array containing the input parameters IN THE ORDER: 'M/F', 'Age', 'Educ', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF'
input_data = np.array([0,49,4,0,29,1564,0.734,1.122])
# NOTE: F=0, M=1 (this took me way too long to fix)
print(predict_cdr(model, input_data)) #Should print close to 0

# more tests
# NOTE: this accuracy may be due to overfitting. Finding new data is instrumental
input_data = np.array([1,82,2,3,27,1420,0.71,1.236])
print(predict_cdr(model, input_data)) #Should print close to .5

input_data = np.array([0,80,3,2,20,1732,0.692,1.013])
print(predict_cdr(model, input_data)) #Should print close to 1

input_data = np.array([1,86,1,4,15,1512,0.665,1.161])
print(predict_cdr(model, input_data)) #Should print close to 2

In [ ]:
def button_click(self, **event_args):
    """This method is called when the button is clicked"""
    # Call the google colab function and pass it the iris measurements
    self.result.text = "Your CDR Prediction is "
    print("worked")
    data = np.array([self.gender.text,self.age.text,self.edu.text,self.ses.text,self.mmse.text,self.etiv.text,self.nwbv.text,self.asf.text]).astype(float)
    result = anvil.server.call('predict_cdr', model, data)

    self.result.visible = True
    self.result.text = "Your CDR Prediction is " + result

In [ ]:
anvil.server.wait_forever()

**Conclusion**

Now, while this may seem like a conclusive Alzheimer's/Dementia detection model, it is far from done. One obvious shortcoming is that if you are left-handed, this model doesn't apply to you at all. There must be serveral other factors like that, and as such it is important that this continues to be studied